#Generative Adversarial Network (GAN)

Código base:
https://github.com/eriklindernoren/Keras-GAN



In [1]:
#Carregar bibliotecas
%tensorflow_version 1.x
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np
import random

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
img_rows = 28 #####
img_cols = 28 #####
channels = 1 #####

input_shape = (img_rows, img_cols, channels) #####
latent_dim = 100
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
# Criar o Gerador
generator = Sequential()

generator.add(Dense(256, input_dim=latent_dim))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod(input_shape), activation='tanh')) #####
generator.add(Reshape(input_shape))
generator.summary()

noise = Input(shape=(latent_dim,))
distribution = generator(noise)
generator = Model(noise, distribution)

# Compilar o gerador
generator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 1024)             

In [4]:
# Criar o Discriminador
discriminator = Sequential()

discriminator.add(Flatten(input_shape=input_shape)) #####
discriminator.add(Dense(512)) #####
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

distribution = Input(shape=input_shape)
validity = discriminator(distribution)
discriminator = Model(distribution, validity)

# Compilar o discriminador
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

In [5]:
# O gerador recebe um ruído como entrada e gera a amostra com a distribuição aprendida
input = Input(shape=(latent_dim,))
gen_data = generator(input)
validity = discriminator(gen_data)

# Para o modelo combinado, somente treinaremos o gerador
discriminator.trainable = False

# O discriminador recebe os dados gerados e determina a validade dos mesmos
validity = discriminator(gen_data)

# Modelo combinado: stacked generator and discriminator
# Treina o gerador para enganar o discriminador
combined = Model(input, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# Mostrar a estrutura do modelo criado
config = combined.get_config()
model = Model.from_config(config)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_1 (Model)              (None, 28, 28, 1)         1493520   
_________________________________________________________________
model_2 (Model)              (None, 1)                 533505    
Total params: 2,027,025
Trainable params: 2,023,441
Non-trainable params: 3,584
_________________________________________________________________


In [6]:
#####
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    plt.subplots(r, c)
    plt.close()
 #####

In [7]:
batch_size=32
epochs=10000
sample_interval=200 #####

#####
# Carrega a base de dados
(X, _), (_, _) = mnist.load_data()
# Normaliza entre -1  e 1
X = X / 127.5 - 1.
X = np.expand_dims(X, axis=3)
#####

# Vetores com 0s e 1s indicando, respectivamente, amostras fakes e reais
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

# ---------------------
#  Treina o Discriminador
# ---------------------
# Seleciona um batch aleatório de dados
  idx = np.random.randint(0, X.shape[0], batch_size) #####
  imgs = X[idx] #####
  noise = np.random.normal(0, 1, (batch_size, latent_dim)) #####

  # Gera um batch de novos dados
  gen_imgs = generator.predict(noise)

  # Treina o discriminador
  d_loss_real = discriminator.train_on_batch(imgs, valid)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ---------------------
  #  Treina o Gerador
  # ---------------------

  # Treina o gerador para que faça o discriminador classificar os dados criados como reais
  g_loss = combined.train_on_batch(noise, valid)

  # Progresso
  print ("%d [D loss: %f, accD.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      sample_images(epoch)


Output hidden; open in https://colab.research.google.com to view.